In [574]:
# Install the Package
!pip install --user reverse_geocoder

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [575]:
# my usual imports and setups
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
#imports downloader
#from getCitiBikeCSV import getCitiBikeCSV

%pylab inline

import os
import sys
#this makes my plots pretty! but it is totally not mandatory to do it
import json
#s = json.load( open(os.getenv ('PUI2016')+"/fbb_matplotlibrc.json") )
#pl.rcParams.update(s)
import datetime as dt

Populating the interactive namespace from numpy and matplotlib


In [576]:
def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [577]:
datestring = '201501'
getCitiBikeCSV(datestring)

file in place, you can continue


In [578]:
df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')

In [579]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [580]:
len(df['end station longitude'])

285552

In [581]:
# To save some runtime, I will play with 200 rows. 
df.sam = df.drop(df.index[:285300])

In [582]:
len(df.sam)

252

In [583]:
df.sam.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
285300,1179,1/31/2015 22:05,1/31/2015 22:25,380,W 4 St & 7 Ave S,40.734011,-74.002939,454,E 51 St & 1 Ave,40.754557,-73.965930,15358,Subscriber,1986.0,1
285301,262,1/31/2015 22:05,1/31/2015 22:09,301,E 2 St & Avenue B,40.722174,-73.983688,428,E 3 St & 1 Ave,40.724677,-73.987834,17979,Subscriber,1983.0,1


In [584]:
df.sam['start station latitude'].dropna(inplace=True)
df.sam['start station longitude'].dropna(inplace=True)

In [585]:
len(df.sam['start station longitude'])

252

In [586]:
len(df.sam['start station latitude'])

252

In [587]:
# Make a new column combining lat and long but in a tuple format
    # Reference: https://stackoverflow.com/questions/16031056/how-to-form-tuple-column-from-two-columns-in-pandas
df.sam['coordinates'] = df.sam[['start station latitude', 'start station longitude']].apply(tuple,axis=1)

In [588]:
df.sam.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,coordinates
285300,1179,1/31/2015 22:05,1/31/2015 22:25,380,W 4 St & 7 Ave S,40.734011,-74.002939,454,E 51 St & 1 Ave,40.754557,-73.965930,15358,Subscriber,1986.0,1,"(40.73401143, -74.00293877)"
285301,262,1/31/2015 22:05,1/31/2015 22:09,301,E 2 St & Avenue B,40.722174,-73.983688,428,E 3 St & 1 Ave,40.724677,-73.987834,17979,Subscriber,1983.0,1,"(40.72217444, -73.98368779)"


In [600]:
# We will need to use reverse_geocoder for 200 plus times so I will use a for loop.
# rg function reads tuples so we need to make it a tuple.
x = tuple(df.sam['coordinates'])
x[5]

(40.73901691, -74.00263761)

In [590]:
import reverse_geocoder as rg

In [615]:
# Now we use rg function to read coordinates and return the address. 
    #Reference: https://github.com/thampiman/reverse-geocoder
coordinate = x[:252]
results = rg.search(coordinate)
#print(results)

location = []

for i in range (len(results)):
    location.append(results[i]['name'])
df.sam['loc'] = location
df.sam

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,coordinates,loc
285300,1179,1/31/2015 22:05,1/31/2015 22:25,380,W 4 St & 7 Ave S,40.734011,-74.002939,454,E 51 St & 1 Ave,40.754557,-73.965930,15358,Subscriber,1986.0,1,"(40.73401143, -74.00293877)",New York City
285301,262,1/31/2015 22:05,1/31/2015 22:09,301,E 2 St & Avenue B,40.722174,-73.983688,428,E 3 St & 1 Ave,40.724677,-73.987834,17979,Subscriber,1983.0,1,"(40.72217444, -73.98368779)",New York City
285302,205,1/31/2015 22:05,1/31/2015 22:09,2021,W 45 St & 8 Ave,40.759291,-73.988597,447,8 Ave & W 52 St,40.763707,-73.985162,15985,Subscriber,1973.0,1,"(40.75929124, -73.98859651)",Manhattan
285303,2044,1/31/2015 22:06,1/31/2015 22:41,157,Henry St & Atlantic Ave,40.690893,-73.996123,339,Avenue D & E 12 St,40.725806,-73.974225,18276,Subscriber,1965.0,1,"(40.69089272, -73.99612349)",New York City
285304,1450,1/31/2015 22:06,1/31/2015 22:30,479,9 Ave & W 45 St,40.760193,-73.991255,432,E 7 St & Avenue A,40.726218,-73.983799,15391,Subscriber,1953.0,1,"(40.76019252, -73.9912551)",Weehawken
285305,746,1/31/2015 22:06,1/31/2015 22:18,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,487,E 20 St & FDR Drive,40.733143,-73.975739,20860,Subscriber,1984.0,1,"(40.73901691, -74.00263761)",New York City
285306,405,1/31/2015 22:06,1/31/2015 22:13,278,Concord St & Bridge St,40.697666,-73.984764,143,Clinton St & Joralemon St,40.692395,-73.993379,17877,Subscriber,1994.0,2,"(40.69766564, -73.98476437)",New York City
285307,179,1/31/2015 22:06,1/31/2015 22:09,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,116,W 17 St & 8 Ave,40.741776,-74.001497,19855,Subscriber,1958.0,1,"(40.73901691, -74.00263761)",New York City
285308,1260,1/31/2015 22:07,1/31/2015 22:28,223,W 13 St & 7 Ave,40.737815,-73.999947,488,W 39 St & 9 Ave,40.756458,-73.993722,19823,Subscriber,1993.0,1,"(40.73781509, -73.99994661)",New York City
285309,538,1/31/2015 22:07,1/31/2015 22:16,500,Broadway & W 51 St,40.762288,-73.983362,422,W 59 St & 10 Ave,40.770513,-73.988038,19622,Subscriber,1987.0,1,"(40.76228826, -73.98336183)",Manhattan
